### 0. Data Preprocessing


*   KDD: https://pslcdatashop.web.cmu.edu/KDDCup
*   STAT: https://pslcdatashop.web.cmu.edu/DatasetInfo?datasetId=507
*   ASSISTments: https://sites.google.com/view/assistmentsdatamining

In [4]:
!pip3 install pytorch_lightning

In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/爱の日记/ITKT
%ls

Mounted at /content/drive
/content/drive/MyDrive/爱の日记/ITKT
ASSISTments2017/  lightning_logs/   __pycache__/
config.py         model.py          riiid-test-answer-prediction/
data_loader.py    model_saint.pt    utils.py
ITKT.ipynb        multihead_ffn.py


In [6]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [7]:
ASSISTments_dir = "/content/drive/MyDrive/爱の日记/ITKT/ASSISTments2017/anonymized_full_release_competition_dataset.csv"
ASSISTments_raw_df = pd.read_csv(ASSISTments_dir)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# ASSISTments_raw_df[ASSISTments_raw_df['studentId']==954]
len(ASSISTments_raw_df['studentId'].unique())

1709

# New Section

In [9]:
ASSISTments_raw_df.head()

# 1. prune irrelevant cols and then drop rows with null value
cols = ['studentId', 'problemId', 'skill', 'timeTaken', 'hintTotal', 'correct']
sub_cols = []
ASSISTments_df = ASSISTments_raw_df[cols]
ASSISTments_df.dropna(subset=cols, inplace=True)

# 2. Enumerate skill id
ASSISTments_df['skill'], _ = pd.factorize(ASSISTments_df['skill'], sort=True)
ASSISTments_df['skill'] += 1

# 4. Type convert
ASSISTments_df['problemId'] = ASSISTments_df['problemId'].astype('int64')
ASSISTments_df['skill'] = ASSISTments_df['skill'].astype('category')
ASSISTments_df['hintTotal'] = ASSISTments_df['hintTotal'].astype('int8')
ASSISTments_df['correct'] = ASSISTments_df['correct'].astype('int8')

# 3. Group by student_id
CONFIG_MAX_SEQ = 500 # we only build model for students who answered no less than 500 questions
ASSISTments_student_log = ASSISTments_df.groupby('studentId').filter(lambda q: len(q) > CONFIG_MAX_SEQ).copy()
ASSISTments_student_log = ASSISTments_student_log.groupby('studentId').apply(lambda r: (r.problemId.values, r.skill.values,r.timeTaken.values,r.hintTotal.values,r.correct.values))

# 4. calculate features
num_of_skills = max(ASSISTments_df['skill'])
num_of_problems = max(ASSISTments_df['problemId'])
max_hint_use = max(ASSISTments_df['hintTotal'])
max_time_use = max(ASSISTments_df['timeTaken'])


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [10]:
ASSISTments_student_log.shape

(750,)

In [11]:
stu = ASSISTments_student_log[598]
print("num_of_problems: ", num_of_problems)
print("num_of_skills: ", num_of_skills)
print("max_hint_use: ", max_hint_use)
print("max_time_use: ", max_time_use)

# print a sample student input
print("==================================================")
print("id for each question: ", stu[0])
print("--------------------------------------------------")
print("skill taken for each question: ", stu[1])
print("--------------------------------------------------")
print("time use for each question: ", stu[2])
print("--------------------------------------------------")
print("hint for each question: ", stu[3])
print("--------------------------------------------------")
print("correctness for each question: ", stu[4])
print("==================================================")

num_of_problems:  22761
num_of_skills:  102
max_hint_use:  56
max_time_use:  9999.0
id for each question:  [2705 2702 2703 ... 9374 9374 9374]
--------------------------------------------------
skill taken for each question:  [27, 27, 27, 27, 27, ..., 75, 75, 75, 75, 75]
Length: 2024
Categories (102, int64): [1, 2, 3, 4, ..., 99, 100, 101, 102]
--------------------------------------------------
time use for each question:  [175.          57.99999976 111.9999998  ...   0.99999952   1.00000024
   5.        ]
--------------------------------------------------
hint for each question:  [1 0 0 ... 3 3 3]
--------------------------------------------------
correctness for each question:  [0 1 0 ... 0 0 1]


In [12]:
class Student_Data():
  
  def __init__(self, group, n_skills, n_hint, n_time, max_seq = CONFIG_MAX_SEQ):
    self.samples = group
    self.n_skills = n_skills
    self.n_hint = n_hint
    self.n_time = n_time
    self.max_seq = max_seq
    self.data = []

    for qid, skill_id, time, hint ,correct in self.samples:
        if len(qid)>=self.max_seq:
            self.data.extend([(qid[l:l+self.max_seq], skill_id[l:l+self.max_seq], time[l:l+self.max_seq], hint[l:l+self.max_seq], correct[l:l+self.max_seq])\
            for l in range(len(qid)) if l%self.max_seq==0])
        # elif len(qid)<self.max_seq and len(qid)>10:
        #     self.data.append((qid, skill_id, time, hint ,correct))
        else:
            continue
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, idx):

    print("__getitem__ for student ", idx)

    qid, skill_id, time, hint ,correct = self.data[idx]
    seq_len = len(qid)

    q = np.zeros(self.max_seq,dtype=int)
    s = np.zeros(self.max_seq,dtype=int)
    t = np.zeros(self.max_seq,dtype=int)
    h = np.zeros(self.max_seq,dtype=int)
    c = np.zeros(self.max_seq,dtype=int)

    if seq_len>=self.max_seq:
      q[:] = qid[-self.max_seq:]
      s[:] = skill_id[-self.max_seq:]
      t[:] = time[-self.max_seq:]
      h[:] = hint[-self.max_seq:]
      c[:] = correct[-self.max_seq:]
    # else:
    #   q[:] = qid
    #   s[:] = skill_id
    #   t[:] = time
    #   h[:] = hint
    #   c[:] = correct
    
    target_qids = q[1:]
    label = c[1:] 

    input_ids = np.zeros(self.max_seq-1,dtype=int)
    input_ids = q[:-1].copy()

    input_skills = np.zeros(self.max_seq-1,dtype=int)
    input_skills = s[:-1].copy()

    input_rtime = np.zeros(self.max_seq-1,dtype=int)
    input_rtime = t[:-1].copy()

    input_rhint = np.zeros(self.max_seq-1,dtype=int)
    input_rhint = h[:-1].copy()


    input = {"input_ids":input_ids, "input_skills": input_skills, "input_rtime":input_rtime.astype(np.int),"input_rhint":input_rhint}

    return input, target_qids, label 
    

In [13]:
# Split train test sets to train_loader and val_loader

from sklearn.model_selection import train_test_split 
import config
import gc # use for free memory

CONFIG_BATCH_SIZE = 32
train, val = train_test_split(ASSISTments_student_log,test_size=0.2) 
print("train size: ",train.shape,"validation size: ",val.shape)
train_dataset = Student_Data(train.values,n_skills=num_of_skills, n_hint=max_hint_use, n_time=max_time_use, max_seq = CONFIG_MAX_SEQ)
val_dataset = Student_Data(val.values,n_skills=num_of_skills, n_hint=max_hint_use, n_time=max_time_use, max_seq = CONFIG_MAX_SEQ)
train_loader = DataLoader(train_dataset,
                          batch_size=CONFIG_BATCH_SIZE,
                          num_workers=2,
                          shuffle=True)
val_loader = DataLoader(val_dataset,
                          batch_size=CONFIG_BATCH_SIZE,
                          num_workers=2,
                          shuffle=False)

# del train_dataset, val_dataset
# gc.collect()

train size:  (600,) validation size:  (150,)


In [14]:
train_dataset.__getitem__(97) # the id=97 student

__getitem__ for student  97


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


({'input_ids': array([1144, 1145, 1145, 1146, 1146, 1146, 1147, 1147, 1147, 1147, 1147,
         1201, 1202, 1202, 1203, 1127, 1225, 1226, 1226, 1226, 1226, 1227,
         1227, 1227, 1228, 1229, 1229, 1229, 1230, 1204, 1205, 1205, 1205,
         1205, 1205, 1206, 1206, 1206, 1207, 1166, 1167, 1168, 1168, 1166,
         1166, 1167, 1168, 1169, 1169, 1170, 1171, 1172, 1173, 1173, 1074,
         1074, 1135, 1136, 1137, 1137, 1137, 1138, 1138, 1138, 1259, 1260,
         1260, 1261, 1261, 1261, 1261, 1262, 1262, 1262, 1262, 1231, 1232,
         1232, 1232, 1232, 1232, 1232, 1232, 1232, 1232, 1232, 1232, 1232,
         1232, 1232, 1232, 1232, 1232, 1233, 1233, 1233, 1233, 1233, 1215,
         1216, 1216, 1216, 1216, 1217, 1217, 1217, 1217, 1217, 1218, 1218,
         1218, 1077, 1077, 1177, 1178, 1178, 1179, 1179, 1179, 1180, 1180,
         1180, 1180, 1180, 1180, 1180, 1180, 1180, 1177, 1178, 1178, 1179,
         1179, 1179, 1179, 1180, 1180, 1180, 1180, 1180, 1180, 1180, 1180,
         118

Data Preprocessing Finished!

In [15]:
!ls

ASSISTments2017  ITKT.ipynb	 model_saint.pt    riiid-test-answer-prediction
config.py	 lightning_logs  multihead_ffn.py  utils.py
data_loader.py	 model.py	 __pycache__


In [16]:
# utils.py

import torch
import numpy as np
from torch import nn
import copy

def ut_mask(seq_len):
    """ Upper Triangular Mask
    """
    return torch.triu(torch.ones(seq_len,seq_len),diagonal=1).to(dtype=torch.bool)

def lt_mask(seq_len):
    """ Lower Triangular Mask
    """
    return torch.tril(torch.ones(seq_len,seq_len),diagonal=-1).to(dtype=torch.bool)

def pos_encode(seq_len):
    """ position Encoding
    """
    return torch.arange(seq_len).unsqueeze(0)

def get_clones(module, N):
    """ Cloning nn modules
    """
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [17]:
# multihead_ffn.py
from torch import nn

#   MultiHead(Qin,Kin,Vin) = Concat(head1,··· ,headh)WO
class FFN(nn.Module):
  def __init__(self,features):
    super(FFN,self).__init__()
    self.layer1 = nn.Linear(features, features)
    self.layer2 = nn.Linear(features, features)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    
  def forward(self, x):
    out = self.drop(self.relu(self.layer1(x)))
    out = self.layer2(out)
    return out

class MultiHeadWithFFN(nn.Module):
  def __init__(self,n_heads,n_dims,mask_type="ut",dropout=0.2):
  # def __init__(self,n_heads,n_dims,mask_type="lt",dropout=0.2):
    super(MultiHeadWithFFN,self).__init__()
    self.n_dims = n_dims
    self.mask_type = mask_type
    self.multihead_attention = nn.MultiheadAttention(embed_dim = n_dims,
                                                      num_heads = n_heads,
                                                        dropout = dropout)
    self.layer_norm1 = nn.LayerNorm(n_dims)
    self.ffn = FFN(features = n_dims)
    self.layer_norm2 = nn.LayerNorm(n_dims)


  def forward(self,q_input,kv_input):
    q_input = q_input.permute(1,0,2)
    kv_input = kv_input.permute(1,0,2)
    query_norm = self.layer_norm1(q_input)
    kv_norm = self.layer_norm1(kv_input)
    if self.mask_type=="ut":
      mask = ut_mask(q_input.size(0))
    else: 
      mask = lt_mask(q_input.size(0))
    # since we have already run it on cuda, omit the following two lines
    # if config.device == "cuda":
    #   mask = mask.cuda()
    out_atten , weights_attent = self.multihead_attention(query=query_norm,
                                                key = kv_norm,
                                                value = kv_norm,
                                                attn_mask = mask)
    out_atten +=  query_norm
    out_atten = out_atten.permute(1,0,2)
    output_norm = self.layer_norm2(out_atten)
    output = self.ffn(output_norm)
    return output + output_norm 

In [18]:
INPUT_EMBEDDING = []

In [23]:
# saint.py (model)

import torch
from torch import nn

class SAINT(nn.Module):
    def __init__(self,n_encoder,n_decoder,enc_heads,dec_heads,n_dims,total_ex,total_skill,total_hint,total_time,total_responses,seq_len):
        super(SAINT,self).__init__()
        self.n_encoder = n_encoder
        self.n_decoder = n_decoder
        self.enocder = get_clones(EncoderBlock(enc_heads,n_dims,total_ex,total_skill,total_hint,total_time,seq_len),n_encoder)
        self.decoder = get_clones(DecoderBlock(dec_heads,n_dims,total_responses,seq_len),n_decoder)
        self.fc = nn.Linear(n_dims,1)
    
    def forward(self,in_exercise,in_skill,in_time,in_hint,in_response):
        first_block = True
        for n in range(self.n_encoder):
          if n>=1:
            first_block=False
          
          enc = self.enocder[n](in_exercise,in_skill,in_time,in_hint,first_block=first_block)
          in_exercise = enc
          in_skill = enc
          in_time = enc
          in_hint = enc

        first_block = True
        for n in range(self.n_decoder):
          if n>=1:
            first_block=False
          dec = self.decoder[n](in_response,encoder_output=in_exercise,first_block=first_block)
          in_exercise = dec
          in_response = dec
          
        return torch.sigmoid(self.fc(dec))


class EncoderBlock(nn.Module):
  def __init__(self,n_heads,n_dims,total_ex,total_skill,total_time,total_hint,seq_len):
    super(EncoderBlock,self).__init__()
    self.seq_len = seq_len
    self.exercise_embed = nn.Embedding(total_ex,n_dims)
    self.skill_embed = nn.Embedding(total_skill,n_dims)
    self.time_embed = nn.Embedding(total_time,n_dims)
    self.hint_embed = nn.Embedding(total_hint,n_dims)
    self.position_embed = nn.Embedding(seq_len,n_dims)
    self.layer_norm = nn.LayerNorm(n_dims)
    
    self.multihead = MultiHeadWithFFN(n_heads=n_heads,
                                            n_dims = n_dims)
  
  def forward(self,input_e,input_skill,input_time,input_hint,first_block=True):
    if first_block:
      _exe = self.exercise_embed(input_e)
      _skill = self.skill_embed(input_skill)
      _time = self.time_embed(input_time)
      _hint = self.hint_embed(input_hint)
      # position_encoded = pos_encode(self.seq_len-1).cuda()
      position_encoded = pos_encode(self.seq_len-1)
      _pos = self.position_embed(position_encoded)
      out = _skill + _exe + _pos
    else:
      out = input_e
    output = self.multihead(q_input=out,kv_input=out)
    return output


class DecoderBlock(nn.Module):
    def __init__(self,n_heads,n_dims,total_responses,seq_len):
      super(DecoderBlock,self).__init__()
      self.seq_len = seq_len
      self.response_embed = nn.Embedding(total_responses,n_dims)
      self.position_embed = nn.Embedding(seq_len,n_dims)
      self.layer_norm = nn.LayerNorm(n_dims)
      self.multihead_attention = nn.MultiheadAttention(embed_dim=n_dims,
                                            num_heads = n_heads,
                                            dropout = 0.2)
      self.multihead = MultiHeadWithFFN(n_heads=n_heads,
                                              n_dims = n_dims)

    def forward(self,input_r,encoder_output,first_block=True):
      if first_block:
        _response = self.response_embed(input_r)
        position_encoded = pos_encode(self.seq_len-1)
        # _pos = self.position_embed(position_encoded.cuda())
        _pos = self.position_embed(position_encoded)
        out = _response + _pos
      else:
        out = input_r      
      out = out.permute(1,0,2)    
      #assert out_embed.size(0)==n_dims, "input dimention should be (seq_len,batch_size,dims)"
      out_norm = self.layer_norm(out)
      # mask = ut_mask(out_norm.size(0))
      mask = lt_mask(out_norm.size(0))
      # if device == "cuda":
      #   mask = mask.cuda()
      out_atten , weights_attent = self.multihead_attention(query=out_norm,
                                                  key = out_norm,
                                                  value = out_norm,
                                                  attn_mask = mask)
      out_atten +=  out_norm
      out_atten = out_atten.permute(1,0,2)
      output = self.multihead(q_input=out_atten,kv_input=encoder_output)
      return output

In [24]:
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score


class SAKTModel(pl.LightningModule):
  def __init__(self,model_args,model="saint"):
    super().__init__()
    if model == "saint":
      self.model = SAINT(**model_args)
      
  def forward(self,exercise,skill,time,hint,response):
    return self.model(exercise,skill,time,hint,response)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=1e-3)
  
  def training_step(self,batch,batch_idx):
    inputs,target_ids,target = batch
    output = self(inputs["input_ids"],inputs["input_skills"],inputs["input_rtime"], inputs["input_rhint"],target_ids) # input_rhint
    target_mask = (target_ids != 0)
    output = torch.masked_select(output.squeeze(),target_mask)
    target = torch.masked_select(target,target_mask)
    loss = nn.BCEWithLogitsLoss()(output.float(),target.float())
    y_pred = output.detach().float().round()
    y_real = target.detach().numpy()
    auc = roc_auc_score(y_real,y_pred)
    self.log("loss", loss)
    self.log("auc", auc)
    return {"loss":loss,"auc":auc,"output":output,"target":target}
  
  def validation_step(self,batch,batch_idx):
    inputs,target_ids,target = batch
    output = self(inputs["input_ids"],inputs["input_skills"],inputs["input_rtime"], inputs["input_rhint"],target_ids)
    target_mask = (target_ids != 0)
    output = torch.masked_select(output.squeeze(),target_mask)
    target = torch.masked_select(target,target_mask)
    loss = nn.BCEWithLogitsLoss()(output.float(),target.float())
    y_pred = output.detach().float().round()
    y_real = target.detach().numpy()
    auc = roc_auc_score(y_real,y_pred)
    self.log("val_loss", loss)
    self.log("val_auc", auc)
    return {"val_loss":loss,"val_auc":auc,"output":output,"target":target}
    

In [21]:
ARGS = {"n_dims": 128,
            'n_encoder': 6,
            'n_decoder': 6,
            'enc_heads': 8,
            'dec_heads': 8,
            'total_ex': 22761,
            'total_skill': 112,
            'total_hint': 56,
            'total_time': 10000,
            'total_responses': 22761,
            'seq_len': CONFIG_MAX_SEQ}

In [22]:
########### TRAINING AND SAVING MODEL #######
checkpoint = ModelCheckpoint(filename="{epoch}_{step}_model",
                              verbose=True,
                              save_top_k=1,
                              monitor="val_auc")

saint_model = SAKTModel(model="saint",model_args=ARGS)
trainer = pl.Trainer(progress_bar_refresh_rate=21,
                      max_epochs=1,callbacks=[checkpoint]) 
trainer.fit(saint_model, train_loader, val_loader) 
trainer.save_checkpoint("model_saint.pt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=21)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1815: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,

  | Name  | Type  | Params
------------------------

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


__getitem__ for student  0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


__getitem__ for student  1
__getitem__ for student  2
__getitem__ for student  32
__getitem__ for student  3
__getitem__ for student  4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


__getitem__ for student  33
__getitem__ for student  34
__getitem__ for student  5
__getitem__ for student  35
__getitem__ for student  36
__getitem__ for student  37
__getitem__ for student  38
__getitem__ for student  6
__getitem__ for student  7
__getitem__ for student  39
__getitem__ for student  40
__getitem__ for student  8
__getitem__ for student  9
__getitem__ for student  41
__getitem__ for student  42
__getitem__ for student  10
__getitem__ for student  43
__getitem__ for student  11
__getitem__ for student  44
__getitem__ for student  45
__getitem__ for student  12
__getitem__ for student  46
__getitem__ for student  47
__getitem__ for student  13
__getitem__ for student  48
__getitem__ for student  14
__getitem__ for student  49
__getitem__ for student  50
__getitem__ for student  15
__getitem__ for student  16
__getitem__ for student  17
__getitem__ for student  18
__getitem__ for student  19
__getitem__ for student  51
__getitem__ for student  52
__getitem__ for student  

Epoch 0, global step 46: 'val_auc' reached 0.71004 (best 0.71004), saving model to '/content/drive/MyDrive/爱の日记/ITKT/lightning_logs/version_11/checkpoints/epoch=0_step=46_model.ckpt' as top 1
